In [2]:
# Program to prepare high-probability last-date carpet-bombing trade

# ...Program takes only 1-day expiry Puts
# ...Establishes safe-trades with Delta = 1
# ...makes df_lastday.pkl for order processing

# STATUS: Incomplete
# Runtime: 25 sec for 1,000 records

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=3) # kavi TWS live
ib=IB().connect('127.0.0.1', 4001, clientId=3) # kavi IBG live

In [ ]:
%%time
import pandas as pd
import numpy as np
import datetime

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

# Read the main file
df_main = pd.read_pickle(datapath+'df_main.pkl')

# Filters puts out with Delta 1 and RoM > 0
puts_df = df_main.loc[(df_main.Type == 'p') & (df_main.Delta == 1) & (df_main.RoM > 0), :]

# mask for columns
put_cols = ['ibSymbol', 'undPrice', 'Strike', 'Expiry', 'DTE', 'pLTP', 'pBidPrice', 
            'Type', 'Mlot', 'Delta', 'RoM']

# Filter out 1-day expiry puts
t_puts = puts_df.loc[puts_df.DTE == 1, put_cols].sort_values(['DTE', 'RoM'], ascending=False)

# Compute price variance percentage
t_puts['price_var'] = (t_puts.pBidPrice - t_puts.pLTP)
t_puts['price_var_pct'] = (t_puts.pBidPrice - t_puts.pLTP)/t_puts.pBidPrice

# Establish target price and weed substitute NaN with bidprice
t_puts['tgt_price'] = np.where(t_puts.price_var < -10, t_puts.pBidPrice+0.15, t_puts.pLTP)
t_puts.loc[t_puts.tgt_price.isna(), 'tgt_price'] = t_puts.pBidPrice + 0.15

# Qualify the contracts, make orders and put in dataframe
c_list = [Option(symbol=symbol, exchange='NSE', 
       lastTradeDateOrContractMonth='20181129', strike=str(strike), right='P') 
for symbol, strike in zip(t_puts.ibSymbol, t_puts.Strike)]

contracts = [c for i in range(0, len(c_list), 50) for c in ib.qualifyContracts(*c_list[i: i+50])]
t_puts['Contracts'] = contracts

t_puts['Orders'] = [LimitOrder(action='SELL', totalQuantity=mlot, lmtPrice=price) 
for mlot, price in zip(t_puts.Mlot, t_puts.tgt_price)]

# Check the data
t_puts.head()

# Pickle for placing the orders
if not t_puts.empty:
    t_puts.to_pickle(datapath+r'df_last_day_orders.pkl')
    t_puts.to_excel(datapath+'df_last_day_orders.xlsx', header=1, freeze_panes=(2,1))

# Disconnect the session
ib.disconnect()